In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ["WANDB_DISABLED"] = "true"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Imports and installations

In [ ]:
#!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

In [ ]:
from datasets import load_dataset, load_metric, Audio
import os
import torch

#set_caching_enabled(False)

minds = load_dataset("superb","asr", split="train")
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds = minds.remove_columns(['speaker_id', 'chapter_id', 'id'])

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]' #ignore some special characters

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

minds = minds.map(remove_special_characters)

In [ ]:
minds = minds.train_test_split(test_size=0.2) #split data into train and test
minds["train"][0]

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocabs = minds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=minds.column_names["train"])

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict) #UNK for unknown
vocab_dict["[PAD]"] = len(vocab_dict) #PAD for padding sequences
print(len(vocab_dict))

In [ ]:
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

## Training model

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
os.mkdir("./tokenizer_dir")

tokenizer.save_pretrained("./tokenizer_dir")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [ ]:
minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=2)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    #print({"wer": wer})
    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2ForCTC

#model = Wav2Vec2ForCTC.from_pretrained(
#    "/kaggle/input/stt-readymade-checkpoint/checkpoint-4000", 
#    ctc_loss_reduction="mean", 
#    pad_token_id=processor.tokenizer.pad_token_id,
#)

#Models can also be provided instead of checkpoints

model = Wav2Vec2ForCTC.from_pretrained(
    "/kaggle/input/stt-full-data-v5/checkpoint-9800", #loading checkpoint
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)



In [ ]:
model.freeze_feature_extractor()

In [ ]:
os.mkdir("./model")

In [ ]:
from transformers import TrainingArguments

#Training args can be changed as required.

training_args = TrainingArguments(
  output_dir="./model",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=8,
  fp16=True,
  gradient_checkpointing=True,
  resume_from_checkpoint="/kaggle/input/stt-full-data-v5/checkpoint-9800",
  save_steps=200,
  eval_steps=500,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer, TrainerCallback

#class PrinterCallback(TrainerCallback):
#    def on_log(self, args, state, control, logs=None, **kwargs):
#        _ = logs.pop("total_flos", None)
#        if state.is_local_process_zero:
#            print(logs)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=minds["train"],
    eval_dataset=minds["test"],
    tokenizer=processor.feature_extractor,
    #callbacks=[PrinterCallback]
)

In [ ]:
trainer.train(resume_from_checkpoint="/kaggle/input/stt-full-data-v5/checkpoint-9800") #checkpoint needs to be provided to continue training
#trainer.train()

## Testing

In [3]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

#Loading models
model = AutoModelForCTC.from_pretrained("/kaggle/input/stt-full-data-test-model/model")
processor = Wav2Vec2Processor.from_pretrained("/kaggle/input/stt-full-data-test-model/model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from datasets import load_dataset, load_metric, Audio

minds = load_dataset("PolyAI/minds14",name="en-US", split="train[:100]")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset minds14 downloaded and prepared to /root/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/aa40414f15e0f919231d617440192034af844835dc1e6a697f4b552e0551fd26. Subsequent calls will reuse this data.


In [10]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds[8]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602baa0fbb1e6d0fbce9214f.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602baa0fbb1e6d0fbce9214f.wav',
  'array': array([-3.8957143e-05, -2.0281275e-04, -2.0516709e-04, ...,
         -1.2579626e-02, -1.1464247e-02, -6.2970519e-03], dtype=float32),
  'sampling_rate': 16000},
 'transcription': 'I need to find out if I probably set up a joint account',
 'english_transcription': 'I need to find out if I probably set up a joint account',
 'intent_class': 11,
 'lang_id': 4}

In [11]:
import torch
mind_val = minds[8]
mind_tens = torch.tensor(mind_val["audio"]["array"])
mind_val

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602baa0fbb1e6d0fbce9214f.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602baa0fbb1e6d0fbce9214f.wav',
  'array': array([-3.8957143e-05, -2.0281275e-04, -2.0516709e-04, ...,
         -1.2579626e-02, -1.1464247e-02, -6.2970519e-03], dtype=float32),
  'sampling_rate': 16000},
 'transcription': 'I need to find out if I probably set up a joint account',
 'english_transcription': 'I need to find out if I probably set up a joint account',
 'intent_class': 11,
 'lang_id': 4}

In [12]:
mind_tens_unsq = mind_tens.unsqueeze(0)
mind_tens_unsq.shape

torch.Size([1, 117400])

### Testing on custom audio
The cell below was created only for testing purpose on direct audio files. It consists of resampling the input voice and apply MFCC. Moreover, even though we get 2 channels as input, we only require a single channels for predictions and hence, 1 channel is discarded.

In [26]:
#import torch
#import torchaudio
#import torch.nn as nn

#PATH = "/kaggle/input/audio-1235/Recording_4.wav"

#Load audio as waveform
#waveform, sample_rate = torchaudio.load(PATH)

#Resampling
#new_sample_rate = 16000
#transform = nn.Sequential(
#    torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate),
#    torchaudio.transforms.MFCC(sample_rate=new_sample_rate, n_mfcc=64)
#    )

#channel=0
#mind_tens_unsq = transform(waveform[channel, :].view(1, -1))

In [13]:
with torch.no_grad():
    logits = model(mind_tens_unsq).logits
    pred_ids = torch.argmax(logits, dim=-1)
    decode_result = processor.batch_decode(pred_ids)[0].replace("[PAD]",'') #Replace PAD token with spaces

## Example outputs
As observable, it works more on the basis of pronunciations and accents. Hence, the results are not perfect but certainly good considering that this is an unseen dataset.

In [14]:
print("Expected result: \n")
print(mind_val["transcription"], "\n")
print("Predicted result: \n")
print(decode_result,"\n")

Expected result: 

I need to find out if I probably set up a joint account 

Predicted result: 

ha i need to find out about how to set up a jointo kan please 

